In [8]:
import numpy as np
import matplotlib.pyplot as plt
import csv

# FIT01 for Fuel Cycle Simulation Reactor Depletion
### This Functionality Isolation Test (FIT) is to test MOX fuel fabrication and depletion functionalities in a reactor module given streams of varying plutonium quality (different isotopics). The four streams are:
### `ref, source_1, source_2, and source_3`.

### ORION is used for this analysis, where a MOX LWR fabrication and reactor module is given four streams with varying plutonium stream. 

### The Fuel Loading Module (FLM) is the `Fuel Fabrication` facility in ORION, that uses cross sections and nu-bars to calculate how to mix the fissile and fertile stream. The datasets with `_ff` are ones using fixed fractions, which means that the FLM is not utilized. (DOESN'T WORK)


### The fissile stream is plutonium and americium in different qualities (higher the source number, lower the quality - less fissile Pu isotopes).
### The fertile stream is depleted uranium (0.238% U235).

### Power density parameter in ORION is calculated by dividng the thermal power by the core mass
```
lwr_mox = 3000MW / 72t = 41.6W/g
sfr = 2500 MW / 51.95t = 48.12 W/g
```

In [9]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [10]:
def get_comp_dict(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        isotopes = lines[0]
        isotopes = isotopes.split(',')[1:-1]
        isotopes = [x.split('+')[1] for x in isotopes]
        masses = lines[-1]
        masses = masses.split(',')[1:-1]
        masses = np.array([float(x) for x in masses])
        total_mass = sum(masses)
    
    comp_dict = {}
    for indx, val in enumerate(isotopes):
        comp_dict[val] = masses[indx]
    
    # get rid of oxygen
    comp_dict = {key: value for key, value in comp_dict.items()
                 if 'O' not in key}
    
    # from mass to percentage
    comp_dict = {key: value / sum(comp_dict.values()) * 100 for key, value
                 in comp_dict.items()}

    return comp_dict

In [11]:
def show_fracs(comp_dict, search_list=[]):
    filtered_dict = {}
    for key, val in comp_dict.items():
        if val < 1e-5:
            continue
        if len(search_list) > 1:
            for keyword in search_list:
                if keyword.upper() in key:
                    filtered_dict[key] = val
        else:
            filtered_dict[key] = val
    return filtered_dict

In [12]:
def get_am_pu(filtered_dict):
    am_pu = 0
    for key, val in filtered_dict.items():
        if 'AM' in key or 'PU' in key:
            am_pu += val
    return am_pu

In [13]:
def main(file_list):
    data = {}
    for file in file_list:
        comp_dict = get_comp_dict(file)
#        frac_dict = show_fracs(comp_dict, ['U', 'NP', 'PU', 'AM'])
        frac_dict = show_fracs(comp_dict,['U', 'PU', 'NP', 'AM'])
        am_pu = get_am_pu(frac_dict)
        filename = file.split('/')[-1]
        filename = filename.split('.csv')[0]
        data.update({filename:{'fraction': frac_dict, 'AM+PU': am_pu}})
    return data

In [14]:
from os import listdir
from os.path import isfile, join
file_list = [f for f in listdir('./results') if isfile(join('./results', f))]
file_list =['./results/' + x for x in file_list]

In [15]:
data = main(file_list)

In [16]:
# show all 
def show_data(data, must_word=''):
    for key, val in data.items():
        if must_word not in key:
            continue
        print('=========================')
        print(key)
        for key2, val2 in val.items():
            if key2 == 'fraction':
                for key3, val3 in val2.items():
                    print('%s = %f' %(key3, val3))
            else:
                print('%s = %f' %(key2, val2))
        print('=========================')
        print('\n')


In [17]:
import pandas as pd 

def make_csv(data, file_list, iso_list, outputfile):
    result = np.vstack(iso_list + ['Am+Pu'])
    for file in file_list:
        array = np.zeros(len(iso_list) + 1)
        for key2, val2 in data[file]['fraction'].items():
            if key2 in iso_list:
                indx = iso_list.index(key2)
                array[indx] = val2
        array[-1] = data[file]['AM+PU']
        result = np.hstack((result, np.vstack(array)))
    df = pd.DataFrame(result)
    df.to_csv(outputfile)
        
"""
        if key in file_list:
            array = np.zeros(len(iso_list) + 1)
            for key2, val2 in  val['fraction'].items():
                if key2 in iso_list:
                    indx = iso_list.index(key2)
                    array[indx] = val2
            array[-1] = val['AM+PU']
            result = np.hstack((result, np.vstack(array)))
    df = pd.DataFrame(result)
    df.to_csv(outputfile)
"""

"\n        if key in file_list:\n            array = np.zeros(len(iso_list) + 1)\n            for key2, val2 in  val['fraction'].items():\n                if key2 in iso_list:\n                    indx = iso_list.index(key2)\n                    array[indx] = val2\n            array[-1] = val['AM+PU']\n            result = np.hstack((result, np.vstack(array)))\n    df = pd.DataFrame(result)\n    df.to_csv(outputfile)\n"

In [18]:
# average composition blanket : driver
driver_frac = 0.565
blanket_frac = 1 - driver_frac

sfr_discharge_list = ['sfr_ref_discharge', 'sfr_1_discharge', 'sfr_2_discharge', 'sfr_3_discharge']
isos = list(data[sfr_discharge_list[0]]['fraction'].keys())
for sfrs in sfr_discharge_list:
    blanket_key = sfrs + '_blanket'
    driver_comp = data[sfrs]['fraction']
    blanket_comp = data[blanket_key]['fraction']
    avg_comp = {}
    for iso in isos:
        avg = driver_comp[iso] * driver_frac + blanket_comp[iso] * blanket_frac
        avg_comp[iso] = avg
    avg_ampu = data[sfrs]['AM+PU'] * driver_frac + data[blanket_key]['AM+PU'] * blanket_frac
    data[sfrs+'_avg'] = {'fraction': avg_comp, 'AM+PU': avg_ampu}


KeyError: 'sfr_ref_discharge'

In [ ]:
# read solutions
from numpy import genfromtxt
def read_sol(solution_csv):
    outdict = {}
    array = genfromtxt(solution_csv, delimiter=',', dtype=None)
    for i in range(1, len(array[0])):
        keyname = array[0][i].decode('utf-8')
        outdict[keyname] = {'fraction': {}, 'AM+PU': 0}
        for j in range(len(array)):
            isoname = array[j][0].decode('utf-8')
            if isoname in ['Total', '', 'wt%', 'other HM?', 'detailed fission products?']:
                continue
            isoname = isoname.replace('-','')
            isoname = isoname.upper()
            if isoname == 'AM+PU':
                outdict[keyname]['AM+PU'] = float(array[j][i])
                continue
            outdict[keyname]['fraction'][isoname] = float(array[j][i])
    return outdict

z = read_sol('./solution/mox_lwr_charge_ff.csv')
data['pu_ref_charge_sol'] = z['ref']
data['source_1_charge_sol'] = z['Pu_Source_01']
data['source_2_charge_sol'] = z['Pu_Source_02']
data['source_3_charge_sol'] = z['Pu_Source_03']
z = read_sol('./solution/mox_lwr_discharge_ff.csv')
data['pu_ref_discharge_sol'] = z['ref']
data['source_1_discharge_sol'] = z['Pu_Source_01']
data['source_2_discharge_sol'] = z['Pu_Source_02']
data['source_3_discharge_sol'] = z['Pu_Source_03']
z = read_sol('./solution/sfr_charge_ff.csv')
data['sfr_ref_charge_sol'] = z['ref']
data['sfr_1_charge_sol'] = z['Pu_Source_01']
data['sfr_2_charge_sol'] = z['Pu_Source_02']
data['sfr_3_charge_sol'] = z['Pu_Source_03']
z = read_sol('./solution/sfr_discharge_ff.csv')
data['sfr_ref_discharge_sol'] = z['ref']
data['sfr_1_discharge_sol'] = z['Pu_Source_01']
data['sfr_2_discharge_sol'] = z['Pu_Source_02']
data['sfr_3_discharge_sol'] = z['Pu_Source_03']
diff_dict = {}

In [192]:
print(data['pu_ref_charge_sol'])
print('\n')
hit_list = ['pu_ref_charge', 'source_1_charge', 'source_2_charge', 'source_3_charge',
            'pu_ref_discharge', 'source_1_discharge', 'source_2_discharge', 'source_3_discharge',
            'sfr_ref_charge', 'sfr_1_charge', 'sfr_2_charge', 'sfr_3_charge',
            'sfr_ref_discharge_avg', 'sfr_1_discharge_avg', 'sfr_2_discharge_avg', 'sfr_3_discharge_avg'
            ]
# generate differences
for hit in hit_list:
    sol_key = hit.replace('_avg', '')
    sol_key = sol_key + '_sol'
    diff_dict = {'fraction':{}, 'AM+PU': 0}
    for key, item in data[sol_key]['fraction'].items():
        try:
            # in percentage
            diff = (item - data[hit]['fraction'][key]) / item * 100
        except KeyError:
            if item == 0:
                diff = 0
            else:
                diff = 100
        except:
            print('SOMETHING IS WRONG BOSS')
        diff_dict['fraction'][key] = diff
    am_pu_diff = (data[sol_key]['AM+PU'] - data[hit]['AM+PU']) / data[sol_key]['AM+PU'] * 100
    diff_dict['AM+PU'] = am_pu_diff    
    data[hit.replace('_avg', '') + '_diff'] = diff_dict

print(data['pu_ref_charge'])
print(data['pu_ref_charge_sol'])
print(data['pu_ref_charge_diff'])
print(data['sfr_1_charge_diff'])

{'fraction': {'U235': 0.221, 'U238': 92.775, 'PU238': 0.139, 'PU239': 4.36, 'PU240': 1.576, 'PU241': 0.56, 'PU242': 0.35, 'AM241': 0.019}, 'AM+PU': 7.0}


{'fraction': {'U235': 0.22133980079417928, 'U238': 92.77866649920014, 'PU238': 0.13859987526011228, 'PU239': 4.357496078253529, 'PU240': 1.5749985825012758, 'PU241': 0.5599994960004535, 'PU242': 0.3499996850002835, 'AM241': 0.01889998299001531}, 'AM+PU': 6.999993700005669}
{'fraction': {'U235': 0.221, 'U238': 92.775, 'PU238': 0.139, 'PU239': 4.36, 'PU240': 1.576, 'PU241': 0.56, 'PU242': 0.35, 'AM241': 0.019}, 'AM+PU': 7.0}
{'fraction': {'U235': -0.15375601546573714, 'U238': -0.003952033629896293, 'PU238': 0.28785952509909074, 'PU239': 0.057429397854841094, 'PU240': 0.06354171946220105, 'PU241': 8.999991902529963e-05, 'PU242': 8.999991899357899e-05, 'AM241': 0.5264053157089001}, 'AM+PU': 8.999991900943931e-05}
{'fraction': {'U235': 0.004408470514055685, 'U238': -7.3831546700331766e-06, 'PU238': 6.308643341361737, 'PU239': 6.3591947994

/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


In [199]:

make_csv(data, ['pu_ref_charge', 'pu_ref_charge_diff', 'source_1_charge', 'source_1_charge_diff',
                'source_2_charge', 'source_2_charge_diff', 'source_3_charge', 'source_3_charge_diff'],
         ['U235', 'U238', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'mox_lwr_charge.csv')

make_csv(data, ['pu_ref_discharge', 'pu_ref_discharge_diff', 'source_1_discharge', 'source_1_discharge_diff',
                'source_2_discharge', 'source_2_discharge_diff', 'source_3_discharge', 'source_3_discharge_diff'],
         ['U234', 'U235', 'U236', 'U238', 'NP237', 'PU236', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'mox_lwr_discharge.csv')

make_csv(data, ['sfr_ref_charge', 'sfr_ref_charge_diff', 'sfr_1_charge', 'sfr_1_charge_diff',
                'sfr_2_charge', 'sfr_2_charge_diff', 'sfr_3_charge', 'sfr_3_charge_diff'],
         ['U235', 'U238', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'sfr_charge.csv')

make_csv(data, ['sfr_ref_discharge_avg', 'sfr_ref_discharge_diff', 'sfr_1_discharge_avg', 'sfr_1_discharge_diff',
                'sfr_2_discharge_avg', 'sfr_2_discharge_diff', 'sfr_3_discharge_avg', 'sfr_3_discharge_diff'],
         ['U234', 'U235', 'U236', 'U238', 'NP237', 'PU236', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'sfr_discharge.csv')


make_csv(data, ['sfr_ref_discharge_avg', 'sfr_ref_discharge_diff', 'sfr_1_discharge', 'sfr_1_discharge_blanket',
                'sfr_2_discharge', 'sfr_2_discharge_blanket', 'sfr_3_discharge', 'sfr_3_discharge_blanket'],
         ['U234', 'U235', 'U236', 'U238', 'NP237', 'PU236', 'PU238', 'PU239', 'PU240', 'PU241', 'PU242', 'AM241'],
         'sfr_discharge_only.csv')



In [200]:
# make sure you have 
#documentclass{article}
#usepackage{xcolor,colortbl}

#newcolumntype{a}{>{\columncolor{red}}c}

#in latex file

def csv_to_latex_table(csvfile, sfr_discharge=False):
    with open(csvfile, 'r') as f:
        lines = f.readlines()
        lines = lines[1:]
        #columns = len(lines[0].split(',')) - 1
        #string = '\t\\begin{tabular}{c|' + 'c'*(columns-1) + '}\n'
        if sfr_discharge:
            string = '\t\\begin{tabular}{c|cc|cc|cc|cc}\n'
        else:
            string = '\t\\begin{tabular}{c|ca|ca|ca|ca}\n'
        string += '\t\t\\hline\n'
        if sfr_discharge:
            string += '\t\t\\textbf{wt\%} & \\textbf{ref driver} & \\textbf{ref blanket} & \\textbf{1 driver} & \\textbf{1 blanket} & \\textbf{2 driver} & \\textbf{2 blanket} & \\textbf{3 driver} & \\textbf{3 blanket} \\\ '
        else:
            string += '\t\t\\textbf{wt\%} & \\textbf{ref}  & \\textbf{\% diff} & \\textbf{1} & \\textbf{\% diff} & \\textbf{2} & \\textbf{\% diff} & \\textbf{3} & \\textbf{\% diff}\\\ \n'        
        string += '\t\t\\hline\n'
        for line in lines:
            if line == lines[-1]:
                string += '\t\t\\hline\n'
                string += '\t\t\\hline\n'
            line = line.split(',')[1:]
            string += '\t\t'
            for indx, i in enumerate(line):
                if 'e' in i:
                    e_indx = i.rfind('e')
                    num = i[:5] + i[e_indx:]
                else:
                    num = i[:5]
                if indx == len(line) - 1:
                    string += num.replace('\n','') + ' \\\ \n' 
                else:
                    if indx == 0:
                        string += '\\textbf{' + str(i) + '} & '
                    else:
                        string += num + ' & '
        string += '\t\t\hline \n'
        string += '\t\end{tabular} \n'
        print(string)
    
    with open('../../docs/fit_01/' + csvfile.replace('.csv','.tex'), 'w') as w:
        w.write(string)

In [201]:
csv_to_latex_table('mox_lwr_charge.csv')
csv_to_latex_table('mox_lwr_discharge.csv')
csv_to_latex_table('sfr_charge.csv')
csv_to_latex_table('sfr_discharge.csv')
csv_to_latex_table('sfr_discharge_only.csv', True)


	\begin{tabular}{c|ca|ca|ca|ca}
		\hline
		\textbf{wt\%} & \textbf{ref}  & \textbf{\% diff} & \textbf{1} & \textbf{\% diff} & \textbf{2} & \textbf{\% diff} & \textbf{3} & \textbf{\% diff}\\ 
		\hline
		\textbf{U235} & 0.221 & -0.15 & 0.221 & -0.15 & 0.221 & -0.15 & 0.221 & -0.15 \\ 
		\textbf{U238} & 92.77 & -0.00 & 92.77 & -0.00 & 92.77 & -0.00 & 92.77 & -0.00 \\ 
		\textbf{PU238} & 0.138 & 0.287 & 0.218 & 0.274 & 0.200 & 0.049 & 0.279 & 8.999e-05 \\ 
		\textbf{PU239} & 4.357 & 0.057 & 3.611 & 0.047 & 3.289 & 0.051 & 2.697 & 0.033 \\ 
		\textbf{PU240} & 1.574 & 0.063 & 1.702 & 0.035 & 2.373 & 0.054 & 1.719 & 0.046 \\ 
		\textbf{PU241} & 0.559 & 8.999e-05 & 0.822 & 0.060 & 0.317 & 0.062 & 1.112 & 0.089 \\ 
		\textbf{PU242} & 0.349 & 8.999e-05 & 0.562 & 0.035 & 0.764 & 0.078 & 0.894 & 0.044 \\ 
		\textbf{AM241} & 0.018 & 0.526 & 0.082 & 0.482 & 0.053 & 0.185 & 0.296 & -0.03 \\ 
		\hline
		\hline
		\textbf{Am+Pu} & 6.999 & 8.999e-05 & 6.999 & 8.999e-05 & 6.999 & 8.999e-05 & 6.999 & 8.999

In [196]:
show_data(data, 'sfr_1_discharge_avg')

sfr_1_discharge_avg
RU103 = 0.008470
RU105 = 0.000032
RU106 = 0.047281
EU152 = 0.000155
EU154 = 0.005591
EU155 = 0.009324
EU156 = 0.000127
U234 = 0.012642
U235 = 0.076284
U236 = 0.025905
U237 = 0.000023
U238 = 74.778896
U239 = 0.000096
NP237 = 0.003281
NP239 = 0.013919
PU238 = 0.275623
PU239 = 8.475873
PU240 = 4.797605
PU241 = 0.659948
PU242 = 1.556201
AM241 = 0.194925
AM242 = 0.000069
AM243 = 0.202652
RU100 = 0.026597
RU101 = 0.244577
RU102 = 0.305550
RU104 = 0.270880
EU151 = 0.000605
EU153 = 0.023097
AM+PU = 16.162976


